In [13]:
import pandas as pd
from lazypredict.Supervised import LazyRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import *

In [2]:
X_train = pd.read_csv('/workspaces/ML-Algorithm/data/processed/X_train.csv')
X_train.head(3)

,age,job,marital,education,default,contact,month,duration,campaign,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed
0,-0.60,-1.00,1.00,-0.40,0.00,0.00,-0.67,-0.76,3.00,0.00,0.00,-0.91,-0.93,-0.70,-0.98,-0.71
1,-0.20,-0.50,1.00,0.60,0.00,-1.00,-0.67,-0.20,-0.50,0.00,0.00,0.00,0.27,0.86,0.00,0.00
2,0.80,0.00,0.00,-0.60,0.00,0.00,-0.33,-0.20,1.00,1.00,1.00,-0.87,0.33,0.32,-1.13,-1.55


In [3]:
X_test = pd.read_csv('/workspaces/ML-Algorithm/data/processed/X_test.csv')
X_test.head(3)

,age,job,marital,education,default,contact,month,duration,campaign,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed
0,-0.53,-1.00,1.00,-0.40,0.00,0.00,-0.67,0.37,0.00,0.00,0.00,-0.91,-1.07,-0.70,-0.97,-0.71
1,0.33,1.50,0.00,0.60,0.00,0.00,1.00,-0.56,-0.50,1.00,1.00,-0.38,-0.74,-0.03,-0.20,0.04
2,-0.93,0.00,0.00,0.00,0.00,0.00,2.00,-0.01,-0.50,0.00,0.00,-0.91,-0.87,-0.84,-0.95,-0.71


In [4]:
y_train = pd.read_csv('/workspaces/ML-Algorithm/data/processed/y_train.csv')
y_train.head(3)

,y
0,0
1,0
2,0


In [5]:
y_test = pd.read_csv('/workspaces/ML-Algorithm/data/processed/y_test.csv')
y_test.head(3)

,y
0,0
1,0
2,0


In [6]:
reg = LazyRegressor()
models, predictions = reg.fit(X_train, X_test, y_train, y_test)

100%|██████████| 42/42 [25:53<00:00, 36.98s/it]   

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002238 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 632
[LightGBM] [Info] Number of data points in the train set: 32938, number of used features: 16
[LightGBM] [Info] Start training from score 0.113425


In [7]:
print(models)

                               Adjusted R-Squared  R-Squared  RMSE  Time Taken
Model                                                                         
GradientBoostingRegressor                    0.41       0.41  0.24        3.40
LGBMRegressor                                0.40       0.40  0.24        0.28
HistGradientBoostingRegressor                0.39       0.40  0.24        0.45
MLPRegressor                                 0.39       0.39  0.24        4.96
RandomForestRegressor                        0.38       0.38  0.25        8.53
XGBRegressor                                 0.36       0.36  0.25        0.29
ExtraTreesRegressor                          0.36       0.36  0.25        4.09
BaggingRegressor                             0.33       0.33  0.26        0.84
BayesianRidge                                0.31       0.32  0.26        0.14
ElasticNetCV                                 0.31       0.32  0.26        0.59
LassoCV                                      0.31   

In [10]:
model_GBR = GradientBoostingRegressor()

model_GBR.fit(X_train, y_train)

GradientBoostingRegressor()

In [11]:
y_pred = model_GBR.predict(X_test)
y_pred

array([ 0.05143855, -0.00292729,  0.19988195, ...,  0.74499067,
       -0.00525748,  0.02434682])

In [15]:
print(f"MSE: {mean_squared_error(y_test, y_pred)}")
print(f"R2 Score: {r2_score(y_test, y_pred)}")

MSE: 0.057202351520398036
R2 Score: 0.41409003707637504


In [16]:
def get_metrics(yhat_test, y_test, yhat_train, y_train):
    # Calcula todas las métricas para el conjunto de entrenamiento
    r2_train = r2_score(y_train, yhat_train)
    mse_train = mean_squared_error(y_train, yhat_train)
    mae_train = mean_absolute_error(y_train, yhat_train)
    medae_train = median_absolute_error(y_train, yhat_train)
    rmse_train = mean_squared_error(y_train, yhat_train, squared=False)
    
    # Calcula todas las métricas para el conjunto de prueba
    r2_test = r2_score(y_test, yhat_test)
    mse_test = mean_squared_error(y_test, yhat_test)
    mae_test = mean_absolute_error(y_test, yhat_test)
    medae_test = median_absolute_error(y_test, yhat_test)
    rmse_test = mean_squared_error(y_test, yhat_test, squared=False)
    
    # Calcula la diferencia entre las métricas de entrenamiento y prueba
    r2_diff = r2_train - r2_test
    mse_diff = mse_train - mse_test
    mae_diff = mae_train - mae_test
    medae_diff = medae_train - medae_test
    rmse_diff = rmse_train - rmse_test
    
    # Crea el DataFrame con todas las métricas
    metrics_data = {
        'R2': [r2_train, r2_test, r2_diff],
        'MSE': [mse_train, mse_test, mse_diff],
        'MAE': [mae_train, mae_test, mae_diff],
        'MedAE': [medae_train, medae_test, medae_diff],
        'RMSE': [rmse_train, rmse_test, rmse_diff]
    }
    
    return pd.DataFrame(data=metrics_data, index=['Train set', 'Test set', 'Difference'])

In [18]:
yhat_train = model_GBR.predict(X_train)
yhat_test = model_GBR.predict(X_test)

In [19]:
get_metrics(yhat_test, y_test, yhat_train, y_train)

,R2,MSE,MAE,MedAE,RMSE
Train set,0.46,0.05,0.12,0.01,0.23
Test set,0.41,0.06,0.12,0.02,0.24
Difference,0.05,-0.00,-0.00,-0.00,-0.01


Los resultados muestran lo siguiente:

- R2 Score: El coeficiente de determinación (R2) indica que el modelo explica aproximadamente el 46% de la variabilidad en los datos de entrenamiento y alrededor del 41% en los datos de prueba. Un valor de 0.46 en los datos de entrenamiento y 0.41 en los datos de prueba sugiere que el modelo está capturando una cantidad significativa de la variabilidad en los datos, pero aún queda algo por explicar.

- Mean Squared Error (MSE): El error cuadrático medio (MSE) es una medida de la calidad de las predicciones del modelo. En este caso, el MSE es de aproximadamente 0.05 en los datos de entrenamiento y 0.06 en los datos de prueba. Cuanto menor sea el valor del MSE, mejor será el rendimiento del modelo, lo que sugiere que el modelo tiene un rendimiento razonablemente bueno en términos de la precisión de las predicciones.

- Mean Absolute Error (MAE): El error absoluto medio (MAE) mide la magnitud promedio de los errores en las predicciones del modelo. Aquí, el MAE es de aproximadamente 0.12 tanto en los datos de entrenamiento como en los de prueba. Cuanto menor sea el valor del MAE, mejor será el rendimiento del modelo.

- MedAE (Median Absolute Error): El MedAE es la mediana de los errores absolutos entre las predicciones del modelo y los valores verdaderos. Aquí, el MedAE es de aproximadamente 0.01 en los datos de entrenamiento y 0.02 en los de prueba.

- Root Mean Squared Error (RMSE): El RMSE es otra medida de la calidad de las predicciones del modelo. En este caso, el RMSE es de aproximadamente 0.23 en los datos de entrenamiento y 0.24 en los de prueba. Al igual que el MSE, cuanto menor sea el valor del RMSE, mejor será el rendimiento del modelo.

En resumen, los resultados sugieren que el modelo tiene un rendimiento razonablemente bueno, con una capacidad decente para explicar la variabilidad en los datos y realizar predicciones precisas. 